<a href="https://colab.research.google.com/github/GarimaChopra/Generative_AI/blob/main/hf_transformers_ch1_ex1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Chapter 1 : Hello Transformer**

Below example is for Sentimental Analysis. It takes an madeup customer feedback and tries to analyse it as Positive or Negative feedback.In this example we are going to deploy a language model to Model Zoo with HuggingFace Transformers and use it to generate an original passage of text in response to the customer feedback


In [2]:
# Allows us to work with Hugging Face Transformers
!pip install pipwin
!pipwin install fiona
!pip install geopandas
!pip install modelzoo-client[transformers]


  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 13.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pipwin: filename=pipwin-0.5.2-py2.py3-none-any.whl size=8769 sha256=c6b9f9b5bbaebcbbf3000679bd120e2a44cc4b798ca72bb01989dbc9ee3889f1
  Stored in directory: /root/.cache/pip/wheels/b4/2c/53/c5a91c548b9f030b592608c24efda23ff966b1cceac6414765
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13705 sha256=b4a9522434bdf34d5b3a0c67d83965857f8936a4f4258805f3a86b019f454136
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
  Created wheel for pyjsparser: filename=pyjsparser-2.7.1-py3-none-any.whl size=25982 sha256=9ba5899fa22e900e42b42ba3ad513136dfdcbe7bcfa08a06d17db0b7337d2b86
  Stored in directory: /root/.cache/pip/wheels/5e/81/26/5956478df303e2bf5a85a5df595bb307bd25948a4bab69f7c7
Successfully

In [3]:
# If need to perform translation

!pip install datasets transformers[sentencepiece]


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 36.1 MB/s eta 0:00:00


Instantiate a pipeline by calling the pipeline() function and providing the name of the task we are interested in which is Text-Classification. By default, the text-classification pipeline uses a model that’s designed for sentiment analysis, but it also supports multiclass and multilabel classification.

In [4]:
# Hugging Face transformers

from transformers import pipeline


In [5]:
classifier = pipeline("text-classification", model="distilbert-base-uncased-finetuned-sst-2-english")

In [6]:
import pandas as pd


Below is the input text complain

In [7]:
text = """Dear Amazon, last week I ordered an Optimus Prime action figure
from your online store in Germany. Unfortunately, when I opened the package,
I discovered to my horror that I had been sent an action figure of Megatron
instead! As a lifelong enemy of the Decepticons, I hope you can understand my
dilemma. To resolve the issue, I demand an exchange of Megatron for the
Optimus Prime figure I ordered. Enclosed are copies of my records concerning
this purchase. I expect to hear from you soon. Sincerely, Bumblebee."""

**SENTIMENT ANALYSIS**

> Checks on the feedback, if it is positive or negative. This task is called sentiment analysis



In [8]:

outputs = classifier(text)
pd.DataFrame(outputs)

,label,score
0,NEGATIVE,0.901546


The sentiment is negative which is justified as it is  a complain.

**NER TAGGING**

> Now using Name Entity Recognition , we can verify if the complain is for service or a product.In NLP, real-world objects like products, places, and people are called named entities, and extracting them from text is called named entity recognition (NER).








In [9]:
ner_tagger = pipeline("ner", aggregation_strategy="simple", model="dbmdz/bert-large-cased-finetuned-conll03-english")
outputs = ner_tagger(text)
pd.DataFrame(outputs)

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


,entity_group,score,word,start,end
0,ORG,0.879010,Amazon,5,11
1,MISC,0.990859,Optimus Prime,36,49
2,LOC,0.999755,Germany,90,97
3,MISC,0.556571,Mega,208,212
4,PER,0.590256,##tron,212,216
5,ORG,0.669692,Decept,253,259
6,MISC,0.498349,##icons,259,264
7,MISC,0.775362,Megatron,350,358
8,MISC,0.987854,Optimus Prime,367,380
9,PER,0.812096,Bumblebee,502,511


The pipeline detected all the entities and also assigned a category such as ORG (organization), LOC (location), or PER (person) to each of them.

**QUESTION ANSWERING**

> In question answering, we provide the model with a passage of text called the context, along with a question whose answer we’d like to extract.



In [14]:
reader = pipeline("question-answering", model="distilbert-base-cased-distilled-squad")
question = "What does the customer want?"
outputs = reader(question=question, context=text)
pd.DataFrame([outputs])

,score,start,end,answer
0,0.631292,335,358,an exchange of Megatron


 The pipeline returns the start and end integers that correspond to the character indices where the answer span was found along with the answer

 **SUMMARIZATION**

> The goal of text summarization is to take a long text as input and generate a short version with all the relevant facts.



In [15]:
summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")
outputs = summarizer(text,  max_length=60, clean_up_tokenization_spaces=True)
print(outputs[0]['summary_text'])

 Bumblebee ordered an Optimus Prime action figure from your online store in Germany. Unfortunately, when I opened the package, I discovered to my horror that I had been sent an action figure of Megatron instead. As a lifelong enemy of the Decepticons, I hope you can understand


Model was able to capture the essence of the problem in the summary created

**TRANSLATION**

> Translation pipeline to translate text from a language to another
Below example transaltes text from English to German


In [16]:
!pip install sacremoses
translator = pipeline("translation_en_to_de",
                      model="Helsinki-NLP/opus-mt-en-de")
outputs = translator(text, clean_up_tokenization_spaces=True, min_length=100)
print(outputs[0]['translation_text'])

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 7.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895241 sha256=bca6b2a36c0b506a68c015306827fb15139c1e033a823428b043f46751f94d86
  Stored in directory: /root/.cache/pip/wheels/00/24/97/a2ea5324f36bc626e1ea0267f33db6aa80d157ee977e9e42fb
Successfully built sacremoses
Sehr geehrter Amazon, letzte Woche habe ich eine Optimus Prime Action Figur aus Ihrem Online-Shop in Deutschland bestellt. Leider, als ich das Paket öffnete, entdeckte ich zu meinem Entsetzen, dass ich stattdessen eine Action Figur von Megatron geschickt worden war! Als lebenslanger Feind der Decepticons, Ich hoffe, Sie können mein Dilemma verstehen. Um das Problem zu lösen, Ich fordere einen Austausch von Megatron für die Optimus Prime Figur habe ich bestellt. Eingeschlossen sind Kopien meiner Aufzeichnungen über diesen Kauf. Ich erwarte, von Ihnen bald zu hören. Aufrichtig

**TEXT GENERATION**


> It can be used to provide fast replies like an autocomplete function



In [17]:
generator = pipeline("text-generation", model="gpt2")

response = "Dear Bumblebee, I am sorry to hear that your order was mixed up."
prompt = text + "\n\nCustomer service response:\n" + response
outputs = generator(prompt, max_length=200)

print(outputs[0]['generated_text'])


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Dear Amazon, last week I ordered an Optimus Prime action figure
from your online store in Germany. Unfortunately, when I opened the package,
I discovered to my horror that I had been sent an action figure of Megatron
instead! As a lifelong enemy of the Decepticons, I hope you can understand my
dilemma. To resolve the issue, I demand an exchange of Megatron for the
Optimus Prime figure I ordered. Enclosed are copies of my records concerning
this purchase. I expect to hear from you soon. Sincerely, Bumblebee.

Customer service response:
Dear Bumblebee, I am sorry to hear that your order was mixed up. I hope that you have the appropriate items
to exchange for your original item, and will send the order in the correct order to Bumblefree to be verified as well as an exchange card. Upon completion of the shopping

request,

I will process your offer immediately. In
